In [3]:
import time 
import datetime

def shijian(timeNum):
    timeStamp = float(timeNum)/1000
    ret_datetime = datetime.datetime.fromtimestamp(timeStamp).strftime("%Y-%m-%d %H:%M:%S.%f")
    return ret_datetime

In [10]:
import os
import pandas as pd

dirlist = os.listdir('../CollectedData/BINS/')
print(dirlist)
for i in dirlist:
    data = pd.read_csv('../CollectedData/BINS/'+i)
    time_df = data.iloc[:,0] #获取时间戳的那一列
    bins_df = data.iloc[:,1:-1] #获取bins的列
    data['max'] = bins_df.max(axis=1) # 找到bins中每一行的最大值，并且赋值给max列
    data['time'] = (time_df-time_df[0])/1000
    output = data[['time','max']]
    output.to_csv('../CollectedData/OutBINS/out_'+i, index=False)

['TEST_BINS3.csv', 'TEST_BINS4.csv', 'TEST_BINS5.csv', 'TEST_BINS6.csv']
